In [1]:
import gradio as gr
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from chromadb.config import Settings as StChroma


/home/isaiaszc/inter_projects/personal-RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the questions into a DataFrame

---

the questions currently are in a txt file named `questions.txt`.


In [6]:
import pandas as pd


In [11]:
# Open the file questions.txt
with open("../data/temporary/questions.txt", "r") as file:
    questions = file.read().splitlines()

# load the file by category, the category start without ¿ and end with :
# create a dictionary with the category as key and the questions as value that are
# under the category

categories = {}
category = ""
for question in questions:
    if len(question) > 0 and question[0] != "¿":
        category = question[:-1]
        categories[category] = []
    else:
        # Append if is not empty
        if len(question) > 0:
            categories[category].append(question[1:])
        # categories[category].append(question[1:])


categories


{'Preguntas por ingredientes': ['Qué recetas puedo hacer con calabaza?',
  'Cómo puedo sustituir la harina de trigo en una receta sin perder la textura?',
  'Qué puedo cocinar si solo tengo chile serrano y queso?',
  'Me puedes sugerir una receta que use pavo y piña?',
  'Cómo puedo hacer una receta que no incluya azúcar?'],
 'Preguntas por categorías': ['Qué recetas rápidas puedo preparar para un almuerzo?',
  'Qué postres puedo hacer que sean sin horno?',
  'Me puedes recomendar una receta vegetariana para la cena?',
  'Qué recetas mexicanas clásicas tienes que incluyan mole?',
  'Qué receta fácil puedo hacer con pasta y mariscos?'],
 'Preguntas por preparación': ['Cuál es el tiempo total para preparar helado de nogada?',
  'Me puedes dar los pasos para hacer una horchata de pumpkin spice?',
  'Cómo puedo preparar tacos dorados en una freidora de aire?',
  'Cuáles son las técnicas necesarias para hacer un margarita cristalina?',
  'Cómo puedo hornear correctamente un pastel sin horno

In [16]:
# load the categories dictionary into a dataframe
# donde "Categorie" y "Questions" son los nombres de las columnas
df = pd.DataFrame(
    [(k, v) for k, vals in categories.items() for v in vals],
    columns=["Categorie", "Questions"],
)

df.head()


,Categorie,Questions
0,Preguntas por ingredientes,Qué recetas puedo hacer con calabaza?
1,Preguntas por ingredientes,Cómo puedo sustituir la harina de trigo en una...
2,Preguntas por ingredientes,Qué puedo cocinar si solo tengo chile serrano ...
3,Preguntas por ingredientes,Me puedes sugerir una receta que use pavo y piña?
4,Preguntas por ingredientes,Cómo puedo hacer una receta que no incluya azú...


# Load the LLM

---


In [2]:
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
)

Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

embed_model = Settings.embed_model


In [3]:
chroma_client = chromadb.PersistentClient(
    path="./chroma_recipe", settings=StChroma(allow_reset=True)
)
chroma_collection = chroma_client.get_or_create_collection(name="recipes")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [4]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model,
)


In [5]:
query_engine = index.as_query_engine()


# Test the questions against the LLM


In [18]:
# using the query engine, ask the questions
# and save the answers into new column called "Answers"
df["Answers"] = df["Questions"].apply(lambda x: query_engine.query(x))


In [19]:
df.head()


,Categorie,Questions,Answers
0,Preguntas por ingredientes,Qué recetas puedo hacer con calabaza?,"Puedes hacer ""Calabacitas Con Tocino Y Queso"" ..."
1,Preguntas por ingredientes,Cómo puedo sustituir la harina de trigo en una...,Puedes utilizar harina de linaza o harina de g...
2,Preguntas por ingredientes,Qué puedo cocinar si solo tengo chile serrano ...,"Si solo tienes chile serrano y queso, puedes p..."
3,Preguntas por ingredientes,Me puedes sugerir una receta que use pavo y piña?,No se proporciona una receta específica que in...
4,Preguntas por ingredientes,Cómo puedo hacer una receta que no incluya azú...,"Para hacer una receta que no incluya azúcar, p..."


In [20]:
# save the dataframe into a csv file called "questions_answers.csv"
df.to_csv("../data/temporary/questions_answers.csv", index=False)
